# SARIMA

In [ ]:
# Check package version (Need to do this first because of having to restarting the runtime.)
from packaging import version
import statsmodels
if version.parse(statsmodels.__version__) < version.parse('0.12.1'):
  !pip install statsmodels==0.12.1

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd '/content/drive/MyDrive/Class/Energy Technology and Management/Topic 05 - Project'

In [ ]:
!ls

## Setting up

In [ ]:
#Perform Dickey-Fuller test:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
# kpss_test
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
      kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

# Load data

In [ ]:
import pandas as pd
df_avg = pd.read_csv('data_processed.csv', parse_dates=['datetime'], index_col='datetime')
df_avg.head()

## \[Mod\] Change the frequency of datetime here.
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [ ]:
# Change the frequency of datetime here.
df_avg.index.freq = 'D'

In [ ]:
import matplotlib.pyplot as plt
df_avg.plot(figsize=(10, 3))
plt.show()

# Test for stationariy and seasonality
- ไม่ทำอะไรเลย
```python
df_diff = df_avg
```

- Diff 1 ครั้ง (```d=1```)

```python
df_diff = df_avg.diff(1).dropna()
```

- Diff 1 ครั้ง (```d=1```) และ Diff ด้วย ช่วง 7 หนึ่งครั้ง (```lag=7, D=1```) 

```python
df_diff = df_avg.diff(1).diff(7).dropna()
```

In [ ]:
df_diff = df_avg.diff(1).dropna()
plot_acf(df_diff, lags=30)
plot_pacf(df_diff, lags=30)
fig, ax = plt.subplots(figsize=(10, 3))
df_diff.plot(ax=ax)
plt.show()
adf_test(df_diff)
kpss_test(df_diff)

# Model selection

## \[Mod\] Change the parameters here

ถ้าโมเดลไม่มี Seasonality ให้ใส่ P, Q, D, lag เป็น 0 ให้หมดดังนี้

```python
P = [0]
D = [0]
Q = [0]
lag = [0]
```

In [ ]:
p = [1,2]
d = [1]
q = [1,2]
P = [1,2]
D = [1]
Q = [1]
lag = [7]
params = list(itertools.product(p, d, q, P, D, Q, lag))
print(f"Number of models to test: {len(params)}")

In [ ]:
fit_results = pd.DataFrame()
for param in params:
    pdq = param[0:3]
    PDQL = param[3:7]
    try:
        mod = SARIMAX(df_avg, order=pdq, seasonal_order=PDQL)
        results = mod.fit(method = 'powell',start_params=np.random.random(7))
        data = {'param': pdq, 'param_seasonal': PDQL, 'AIC':results.aic }
        fit_results = fit_results.append(data, ignore_index=True)
    except:
        continue
fit_results = fit_results.sort_values(by='AIC',ascending=True)

In [ ]:
fit_results.head(10)

# Model training

In [ ]:
rank = 1
pdq = fit_results.iloc[rank-1,1]
PDQL = fit_results.iloc[rank-1,2]

print(f"Using ({pdq[0]},{pdq[1]},{pdq[2]})({PDQL[0]},{PDQL[1]},{PDQL[2]},{PDQL[3]})")

mod = SARIMAX(df_avg, order=pdq, seasonal_order=PDQL)
results = mod.fit(method = 'powell', start_params=np.random.random(7))

# Model evaluation

In [ ]:
fig = results.plot_diagnostics(figsize=(10, 6))
fig.tight_layout()

In [ ]:
pred = results.get_prediction(start=df_avg.index[1], end=df_avg.index[-1], dynamic=False)
comb = pd.concat([df_avg, pred.predicted_mean], axis=1).dropna()
comb['error'] = comb.iloc[:,0] - comb.iloc[:,1]
comb['percentage'] = comb['error']/comb.iloc[:,0]*100

MAE = comb['error'].abs().mean()
RMSE = np.sqrt((comb['error']**2).mean())
MAPE = comb['percentage'].abs().mean()

print(f"Mean absolute error: {MAE:6.3f}")
print(f"Root mean squared error: {RMSE:6.3f}")
print(f"Mean absolute percentage error: {MAPE:6.3f}")


## Store preduction results in "df_results" dataframe.

In [ ]:
df_result = pd.concat((df_avg, pred.predicted_mean, pred.conf_int()), axis=1)
df_result.columns = ['y', 'y_pred', 'y_pred_lower', 'y_pred_upper']
df_result = df_result.dropna()
df_result.tail()

# Forecasting

In [ ]:
n_forecast = 10

In [ ]:
# Figure out start and end date
freq = df_avg.index.freq
start_dt = df_avg.index[-1] + freq 
end_dt = start_dt + n_forecast * freq

# Get prediction
pred = results.get_prediction(start=start_dt, end=end_dt, dynamic=False)

df_forecast = pd.concat( (pred.predicted_mean, pred.conf_int()), axis=1)
df_forecast.columns = ['y_pred', 'y_pred_lower', 'y_pred_upper']
df_forecast.index.name = 'datetime'
df_forecast.head()

# Plotting

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

df_result[['y']].plot(ax=ax)
df_result[['y_pred']].plot(ax=ax)
df_forecast[['y_pred']].plot(ax=ax,linestyle='--',marker='s')

dfp = df_result
ax.fill_between(dfp.index, dfp['y_pred_lower'], dfp['y_pred_upper'], color='k', alpha=.1)
dfp = df_forecast
ax.fill_between(dfp.index, dfp['y_pred_lower'], dfp['y_pred_upper'], color='k', alpha=.1)

ax.set_xlabel('Date / Time')
ax.set_ylabel('Y')
ax.legend(['Original','Predicted','Forecasted'])

yp_max = df_result['y'].max()
yp_min = df_result['y'].min()
yp_mean = df_result['y'].mean()
ax.set_ylim(yp_min-0.1*yp_mean,yp_max+0.1*yp_mean)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

df_result[['y']].plot(ax=ax)
df_result[['y_pred']].plot(ax=ax)
df_forecast[['y_pred']].plot(ax=ax,linestyle='--',marker='s')

dfp = df_result
ax.fill_between(dfp.index, dfp['y_pred_lower'], dfp['y_pred_upper'], color='k', alpha=.1)
dfp = df_forecast
ax.fill_between(dfp.index, dfp['y_pred_lower'], dfp['y_pred_upper'], color='k', alpha=.1)

ax.set_xlabel('Date / Time')
ax.set_ylabel('Y')
ax.legend(['Original','Predicted','Forecasted'])

dt_start = df_result.index[-n_forecast*4]
dt_end = df_forecast.index[-1]
ax.set_xlim(dt_start,dt_end)

yp_max = df_result['y'].max()
yp_min = df_result['y'].min()
yp_mean = df_result['y'].mean()
ax.set_ylim(yp_min-0.1*yp_mean,yp_max+0.1*yp_mean)

plt.show()

# Writing data to files

In [ ]:
df_result.to_csv('data_SARIMA_predict.csv')

In [ ]:
df_forecast.to_csv('data_SARIMA_forecast.csv')